In [1]:
import os
os.environ['LOGS_DIRECTORY'] = '../eval_logs'

import sys
sys.path.insert(0, '..')

In [2]:
import json
import random

from tqdm.auto import tqdm
from pydantic import BaseModel
from pydantic_ai import Agent

from main import initialize_index, initialize_agent
from logs import log_interaction_to_file, LOG_DIR

In [3]:
LOG_DIR.absolute()

PosixPath('/home/oliver/Desktop/Agents/ai-agent-crash-course/code/eval/../eval_logs')

In [4]:
index = initialize_index()
agent = initialize_agent(index)

2025-10-04 01:06:23,122 [INFO] aihero: Starting AI Pydantic-AI Assistant for pydantic/pydantic-ai
2025-10-04 01:06:23,123 [INFO] aihero: Initializing data ingestion...
2025-10-04 01:06:26,846 [INFO] aihero: Fetched 114 documents from the repository.
2025-10-04 01:06:26,847 [INFO] aihero: Chunking documents using markdown_sections...
2025-10-04 01:06:26,849 [INFO] aihero: Total chunks created: 606
2025-10-04 01:06:26,850 [INFO] aihero: Indexing documents...
2025-10-04 01:06:26,915 [INFO] aihero: Data indexing completed successfully!
2025-10-04 01:06:26,916 [INFO] aihero: Initializing search agent...
2025-10-04 01:06:26,918 [INFO] aihero: Loading embedding model...
2025-10-04 01:06:26,958 [INFO] sentence_transformers.SentenceTransformer: Use pytorch device_name: cuda:0
2025-10-04 01:06:26,959 [INFO] sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: multi-qa-distilbert-cos-v1
2025-10-04 01:06:34,652 [INFO] aihero: Creating embeddings for vector search...
2025

Creating embeddings: 100%|██████████| 606/606 [00:12<00:00, 50.24it/s] 


2025-10-04 01:06:47,003 [INFO] aihero: Agent initialized successfully!


In [5]:
question_generation_prompt = """
You are helping to create example user questions for an AI assistant that specializes in the **pydantic-ai** repository.

Based on the provided documentation or repository content, generate realistic questions that developers might ask.

The questions should:
- Be natural and varied in phrasing
- Range from beginner-friendly to advanced
- Cover practical usage, installation, integration, features, and troubleshooting
- Include both conceptual and code-related questions
- Focus specifically on the pydantic-ai project

Generate one question for each record.
""".strip()

class QuestionsList(BaseModel):
    questions: list[str]

question_generator = Agent(
    name="question_generator",
    instructions=question_generation_prompt,
    model='groq:openai/gpt-oss-120b',
    output_type=QuestionsList
)

In [7]:
sample = random.sample(index.docs, 10)
prompt = [d['content'] for d in sample]

In [8]:
result = await question_generator.run(user_prompt=json.dumps(prompt))
question_list = result.output

2025-10-04 00:42:06,148 [INFO] httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
# question_list = QuestionsList(questions=['How can I use an ExternalToolset to handle deferred tool calls and feed the results back into a Pydantic‑AI agent?', 'Is it possible to stream structured responses from the model and display them live in a terminal UI (e.g. with Rich) using the StreamedResponse API?', 'What breaking changes were introduced in v0.7.6 of pydantic‑ai and how do I migrate my existing code?', 'In the Data Analyst example, how do I use agent dependencies to pass a pandas DataFrame from one tool to another within the same run?', 'How do I configure retry logic for tools in Pydantic‑AI using the pydantic_ai.retries module?', 'If a tool raises a ModelRetry, how can I ensure the agent automatically retries with a new prompt?', 'How do I set up a FastMCP server that allows sampling from a language model inside a custom tool?', 'Using FunctionModel, how can I mock LLM calls for unit testing a weather‑forecast agent?', 'What’s the correct way to use StateDeps with a custom Pydantic BaseModel to maintain per‑request state in an AG‑UI front‑end?', 'How can I specify an output_type that includes several Pydantic models and a plain string, and what should I keep in mind for type‑checking?'])
question_list

QuestionsList(questions=['How can I use an ExternalToolset to handle deferred tool calls and feed the results back into a Pydantic‑AI agent?', 'Is it possible to stream structured responses from the model and display them live in a terminal UI (e.g. with Rich) using the StreamedResponse API?', 'What breaking changes were introduced in v0.7.6 of pydantic‑ai and how do I migrate my existing code?', 'In the Data Analyst example, how do I use agent dependencies to pass a pandas DataFrame from one tool to another within the same run?', 'How do I configure retry logic for tools in Pydantic‑AI using the pydantic_ai.retries module?', 'If a tool raises a ModelRetry, how can I ensure the agent automatically retries with a new prompt?', 'How do I set up a FastMCP server that allows sampling from a language model inside a custom tool?', 'Using FunctionModel, how can I mock LLM calls for unit testing a weather‑forecast agent?', 'What’s the correct way to use StateDeps with a custom Pydantic BaseMod

In [9]:
for question in tqdm(question_list.questions):
    print(question)
    result = await agent.run(user_prompt=question)
    print(result.output)
    log_interaction_to_file(agent, result.new_messages(), source='ai-generated')
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

How can I use an ExternalToolset to handle deferred tool calls and feed the results back into a Pydantic‑AI agent?
2025-10-04 01:09:21,565 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-04 01:09:33,007 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To use an `ExternalToolset` for handling deferred tool calls and feeding results back into a Pydantic-AI agent, you can follow these steps:

1. **Define External Tools**: Create an external tool that will define what actions to take if the task cannot be completed during the same agent run. This can be achieved by raising the `CallDeferred` exception when certain conditions are met, such as needing longer processing time or waiting for user approval.

2. **Handling Deferred Requests**: When an external tool is called and cannot execute immediately, the agent run will yield a `DeferredToolRequests` object, which contains information about the tool calls. This includes tool names, their arguments, and unique identifiers for each tool call.

3. **Execute the Tasks Externally**: Implement the logic to execute the external task (e.g., using a background worker or an upstream

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-10-04 01:11:19,044 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To mock LLM calls for unit testing a weather-forecast agent using `FunctionModel`, you can follow these steps as illustrated in the relevant documentation:

1. **Define a Mock Function**: Create a function that simulates the LLM call. This function will process incoming messages, extract necessary information (like location and date), and return mock responses.

2. **Use `FunctionModel`**: Utilize `FunctionModel` to replace the agent's default model with your custom function. This allows you to control how the function behaves during the test, making it return predictable results.

Here's an example implementation based on the documentation:

```python
import re
import pytest
from pydantic_ai import models
from pydantic_ai import ModelMessage, ModelResponse, TextPart, ToolCallPart
from pydantic_ai.models.function import FunctionModel, AgentInfo
from fake_database import

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-04 01:11:31,524 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To use `StateDeps` with a custom Pydantic `BaseModel` to maintain per-request state in an AG-UI front-end, you need to follow these steps:

1. **Create a Pydantic BaseModel**: Define a subclass of `BaseModel` which will hold the state you wish to manage for each request. 

2. **Define Your Agent**: Set up your AG-UI agent with the `StateDeps` dependency type, specifying your custom `BaseModel`.

3. **Create an ASGI Application**: Convert your agent to an ASGI application that can handle requests. This can be done using `agent.to_ag_ui()`.

Here's a complete example based on the documentation:

```python
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.ag_ui import StateDeps

class DocumentState(BaseModel):
    """State for the document being written."""
    document: str = ''

# Create the agent with StateDeps for DocumentState
agent = Agent

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-04 01:11:46,046 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To specify an `output_type` that includes multiple Pydantic models and a plain string, you can use a union or a list for defining the output types when initializing the `Agent`. Here's how you can do it:

### Example Code
```python
from pydantic import BaseModel
from pydantic_ai import Agent

class Box(BaseModel):
    width: int
    height: int
    depth: int
    units: str

# Initialize an Agent with output_type including both a Pydantic model and a string
agent = Agent(
    'openai:gpt-4o-mini',  # Specify your model
    output_type=[Box, str],  # Specify the Pydantic model and string type
    system_prompt=(
        "Extract the dimensions of a box. "
        "If unable to extract all data, ask the user to try again."
    ),
)

result = agent.run_sync('The box is 10x20x30')  # Example input
print(result.output)  # Outputs result based on the input
```

### Type-Check